In [1]:
#!/usr/bin/env python3
# _*_ coding: utf-8 _*_

import sys

sys.path.append("../..")

In [2]:
from d2l import d2l_cn as d2l

In [3]:
import time
import math
import numpy as np
import torch
from torch import nn, optim

import torch.nn.functional as F·

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [6]:
def one_hot(x,n_class,dtype=torch.float32):
    x = x.long()
    res = torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    
    res.scatter_(1,x.view(-1,1),1)
    
    return res

In [7]:
x = torch.tensor([0,2])

In [8]:
one_hot(x,vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [9]:
def to_onehot(X,n_class):
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]

In [10]:
X = torch.arange(10).view(2,5)

In [11]:
inputs = to_onehot(X,vocab_size)

In [12]:
print(len(inputs),inputs[0].shape)

5 torch.Size([2, 2582])


In [13]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

In [14]:
def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0,0.01,size=shape),dtype=torch.float32,device=device)
        return torch.nn.Parameter(ts,requires_grad=True)
    
    #隐藏层
    W_xh = _one((num_inputs,num_hiddens))
    W_hh = _one((num_hiddens,num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens,device=device,requires_grad=True))
    
    #output层
    W_hq = _one((num_hiddens,num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs,device=device,requires_grad=True))
    
    return nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])

#### 定义RNN模型

1. 初始化隐藏层状态

In [15]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)

In [25]:
def rnn(inputs,state,params):
    W_xh,W_hh,b_h,W_hq,b_q = params
    H, = state
    outputs = []
    #每个输入和输出是num_steps个形状为(batch_size,vocab_size)的矩阵
    for X in inputs:
        #计算当前时间步的隐藏状态
        H = torch.tanh(torch.matmul(X,W_xh) + torch.matmul(H,W_hh) + b_h)
        Y = torch.matmul(H,W_hq) + b_q
        outputs.append(Y)
    #outputs返回整个前向过程中的输出outputs (num_steps, batch_size, num_outputs)
    #H是最后一个时间步产生的隐藏层状态，(batch_size, num_hiddens)
    return outputs,(H,)

Test:

In [17]:
state = init_rnn_state(X.shape[0],num_hiddens,device)

inputs = to_onehot(X.to(device),vocab_size)

params = get_params()

outputs, state_new = rnn(inputs,state,params)

print(len(outputs),outputs[0].shape,state_new[0].shape)

5 torch.Size([2, 2582]) torch.Size([2, 256])


In [18]:
def predict_rnn(prefix , num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1,num_hiddens,device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix) - 1):
        X = to_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)
        (Y, state) = rnn(X,state,params)
        
        if t < len(prefix) - 1 :
            output.append(char_to_idx[prefix[t+1]])
        
        output.append(int(Y[0].argmax(dim=1).item()))
    
    return ''.join([idx_to_char[i] for i in output])
        

In [19]:
predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'分开契摇片鸟推浮望继托戎障'

In [20]:
print(len('分开'))

2


### 裁剪梯度 clip gradient

对所有参数的梯度，组成向量，裁剪：$$ \min(\frac{\theta}{\|g\|},1) \times \mathcal{g} $$

In [21]:
def grad_clipping(params, theta, device):
    norm  = torch.tensor([0.0],device = device)
    for parm in params:
        norm += (parm.grad.data**2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

#### 困惑度 perplexity

困惑度 = 交叉熵损失函数做指数运算后得到的值

In [22]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, vocab_size, device, corpus_indices, idx_to_char, char_to_idx, is_random_iter,num_epochs, num_steps, lr, clipping_theta, batch_size, pred_peroid, pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    parms = get_params()
    loss = nn.CrossEntropyLoss()
    
    
    for epoch in range(num_epochs):
        #如果不是随机采样，相邻采样
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        
        data_iter = data_iter_fn(corpus_indices,batch_size,num_steps, device)
        
        for X,Y in data_iter:
            # 如使用随机采样，在每个小批量更新前初始化隐藏状态
            if is_random_iter:
                state = init_rnn_state(batch_size,num_hiddens, device)
                 # 否则需要使用detach函数从计算图分离隐藏状态, 这是为了
            # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
            else:
                for s in state:
                    s.detach_()
            inputs = to_onehot(X,vocab_size)
            (outputs, state) = rnn(inputs, state, params)
            outputs = torch.cat(outputs, dim = 0)
            y = torch.transpose(Y,0,1).contiguous().view(-1)
            l = loss(outputs, y.long())#一个批次的交叉熵一起做，计算平均分类误差
            
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
                
            l.backward()
            
            grad_clipping(params, clipping_theta, device)
            d2l.sgd(params, lr , 1)# 因为误差已经取过均值，梯度不用再做平均
            
            l_sum += l.item() * y.shape[0]
            n+=y.shape[0]
            
        
        if (epoch + 1) % pred_peroid == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % ( epoch+1 , math.exp(l_sum/n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))
            

In [23]:
#超参数
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [24]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 40.242179, time 8.87 sec
 - 分开 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不
 - 不分能开再不要这个奖 不想问 我只想要留一口一                                
epoch 100, perplexity 15.056887, time 6.38 sec
 - 分开开始知道 我爱我已经不了  昨事   得飘不想 你说  我不过 我要过 爱不得再不要这样奖 我说你这张
 - 不分能开再要 我不能再想 我不 我不 我不要再想你 爱情 是你弹 你说你 你读我 我不过 你说 为我给我 装在
epoch 150, perplexity 9.538066, time 10.40 sec
 - 分开手 你让我们的爱离开 你靠我的眼泪 让它莹在这条 像什么过去 我去你走到 我我同在哭 一场解 微上的香
 - 不分能开再这样忍我 我知道说这样默会 想要我一口会有人 说不见你 背我的手不能 爱我 缺再是一遍的寂寞 在月中
epoch 200, perplexity 6.654756, time 9.98 sec
 - 分开手 你要过 你不断要想哭 你手不得我 你不要你 这么    我   我看 这里  坚紧 靠a     
 - 不分会开再牵手 这世过 最后的 像我跟枪这种 但一把的梦爱 我用的风 在城中外季袋里 你说不记 我一口 你怎么
epoch 250, perplexity 5.148338, time 11.24 sec
 - 分开手 你怎么 手不是我 你那迷我的痛  你说的爱我想舍在抽美 不要爱你 你已的爱情绪没了很多 一天星大的
 - 不分能开再这样忍 你就笑走你嘴 读多在这里 也让他们 慢慢清背你的不着 我会开地尝一口 你说的爱我 舍不得吃会
